# Stock Market Prediction 



1.   Amazon - AMZN
2.   Apple - AAPL
3.   Microsoft - MSFT
4. Google - GOOGL
5. Tesla - TSLA
6. Oracle - ORCL


In [ ]:
# Dataset download
import pandas_datareader as pr
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error

company = 'ORCL'
price_type = 'close'

data = pr.get_data_tiingo(company, api_key="0f6351ae343427e511f4d085681db7e303ffb969")
data.to_csv(company+'.csv')

data = pd.read_csv(company+'.csv')

# Converting data to timeseries input

def create_dataset(dataset, time_step=1):
  dataX, dataY = [], []
  for i in range(len(dataset)-time_step-1):
    a = dataset[i:(i+time_step), 0]  
    dataX.append(a)
    dataY.append(dataset[i + time_step, 0])
  return np.array(dataX), np.array(dataY)

Prediction = list()

MError = list()
MTrain_size = list()
MTest_size = list()
MTrain_error = list()
MTest_error = list()


for step in [30, 40, 50, 70, 90]:

  # Baseline calculation


  Price = data[price_type]
  Means = list()

  for i in range(len(Price) - step):
    Means.append(sum(Price[i:i+step])/step)  

  Error = math.sqrt(mean_squared_error(Price[step:],Means))

  # MinMax Scalimg
  import numpy as np
  from sklearn.preprocessing import MinMaxScaler

  mms = MinMaxScaler(feature_range=(0,1))
  Price = mms.fit_transform(np.array(Price).reshape(-1,1))

  # Dataset split into train and test sets

  train_size = int(len(Price)*0.80)
  test_size = len(Price)-train_size
  train_data,test_data = Price[0:train_size,:],Price[train_size:len(Price),:1]
  
  # Converting data to timeseries input

  time_step = step
  trainX, trainY = create_dataset(train_data, time_step)
  testX, testY = create_dataset(test_data, time_step)

  trainX = trainX.reshape(trainX.shape[0],trainX.shape[1] , 1)
  testX = testX.reshape(testX.shape[0],testX.shape[1] , 1)

  # Stacked LSTM model

  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import LSTM
  from tensorflow.keras.layers import Dense

  model = Sequential()
  model.add(LSTM(128, return_sequences=True, input_shape= (step, 1)))
  model.add(LSTM(64, return_sequences=False))
  model.add(Dense(25))

  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mean_squared_error')

  #Training the model
  model.fit(trainX,trainY,validation_data=(testX,testY),epochs=100,batch_size=64,verbose=1)

  # Prediction and error calculation

  train_predict = model.predict(trainX)
  test_predict = model.predict(testX)

  testY = testY.reshape(test_size-step-1,1)
  trainY = trainY.reshape(train_size-step-1,1)

  testY = mms.inverse_transform(testY)
  trainY = mms.inverse_transform(trainY)


  train_predict = mms.inverse_transform(train_predict)
  test_predict = mms.inverse_transform(test_predict)


  MError.append(Error)
  MTrain_size.append(train_size-step-1)
  MTest_size.append(test_size-step-1)
  MTrain_error.append(math.sqrt(mean_squared_error(trainY,train_predict)))
  MTest_error.append(math.sqrt(mean_squared_error(testY,test_predict)))
  Prediction.append(test_predict)



In [ ]:
model.summary()
steps = [30, 40, 50, 70, 90]
for i in range(len(steps)):
  print("For step:",steps[i])
  print("Baseline error:",MError[i])
  print("Train size:",MTrain_size[i])
  print("Test size:",MTest_size[i])
  print("Train error:",MTrain_error[i])
  print("Test error:",MTest_error[i])
plt.style.use("fivethirtyeight")  
plt.figure(figsize=(14,8))
plt.title("Different Values of Time Steps")
plt.plot(Prediction[0][-150:],label = "step = 30")
plt.plot(Prediction[1][-150:],label = "step = 40")
plt.plot(Prediction[2][-150:],label = "step = 50")
plt.plot(Prediction[3][-150:],label = "step = 70")
plt.plot(Prediction[4][-150:],label = "step = 90")
plt.plot(testY[-150:],label = "original")
plt.legend()
plt.show()